In [1]:
import pandas as pd
import os
import numpy as np
import re
import matplotlib.pyplot as plt

In [3]:
# Arbeitsverzeichnis ändern 
#neues_verzeichnis = "c:\\Users\\lpres\\OneDrive - JGU\\WMDV\\WebMining"
neues_verzeichnis = "C:\\Users\\Lucian\\Desktop\\WebMining"
os.chdir(neues_verzeichnis)

# Überprüfen, ob das Verzeichnis geändert wurde
print("Neues Arbeitsverzeichnis:", os.getcwd())

# Alle vorhandenen owned_games_x Dateien finden
owned_files = [f for f in os.listdir('.') if re.match(r'owned_games_\d+\.csv', f)]
owned_files.sort(key=lambda x: int(re.search(r'\d+', x).group()))

print(f"Vorhandene Dateien: {owned_files}")

if not owned_files:
    print("Keine Dateien im Format 'owned_games_x.csv' gefunden. Das Skript wird beendet.")

# Alle Dateien einlesen und in einem DataFrame speichern
pre_df = pd.DataFrame()  # Leerer DataFrame

for file in owned_files:
    file_df = pd.read_csv(file, dtype={
         "steam_64_id": str,
         "appid": "Int64",  
         "playtime_forever": "Int64",
         "playtime_2weeks": "Int64"
    })
    pre_df = pd.concat([pre_df, file_df], ignore_index=True)

print("alle Dateien eingelesen")

Neues Arbeitsverzeichnis: C:\Users\Lucian\Desktop\WebMining
Vorhandene Dateien: ['owned_games_1.csv', 'owned_games_2.csv', 'owned_games_3.csv', 'owned_games_4.csv', 'owned_games_5.csv', 'owned_games_6.csv']


C:\Users\Lucian\AppData\Local\Temp\ipykernel_17380\741617666.py:22: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  file_df = pd.read_csv(file, dtype={


alle Dateien eingelesen


In [4]:
# Konvertiere `appid` in einen Integer-Typ (falls möglich) und überprüfe auf fehlende Werte
pre_df['appid'] = pd.to_numeric(pre_df['appid'], errors='coerce', downcast='integer')

combined_data = pre_df.dropna(subset=['steam_64_id', 'appid'])

# Bereinige und aggregiere Daten: Spieler-Zusammenfassung
player_summary = combined_data.groupby('steam_64_id').agg(

    total_playtime=('playtime_forever', 'sum'),

    total_games=('appid', 'count'),

    active_games=('playtime_2weeks', lambda x: (x > 0).sum())

).reset_index()

# Berechne weitere Merkmale
player_summary['avg_playtime_per_game'] = player_summary['total_playtime'] / player_summary['total_games']
player_summary = player_summary.fillna(0)  # Setze NaN-Werte auf 0

# Überblick der bereinigten Daten
print("Bereinigte Spieler-Zusammenfassung (erste Zeilen):")

print(player_summary.head())

Bereinigte Spieler-Zusammenfassung (erste Zeilen):
         steam_64_id  total_playtime  total_games  active_games  \
0                  0               0         4601             0   
1  76561197960359575          135353          158             0   
2  76561197960454727               0           18             0   
3  76561197960481035          747333          275             3   
4  76561197960487929               0          127             0   

   avg_playtime_per_game  
0                    0.0  
1             856.664557  
2                    0.0  
3            2717.574545  
4                    0.0  
